# Свёрточные сети

Нейронные сети привлекли по-настоящему всеобщее внимание в 2012 году, когда Алекс Крижевски благодаря им выиграл конкурс ImageNet — крупнейшее ежегодное соревнование по машинному зрению — снизив рекорд ошибок классификации с 26% до 15%, что тогда стало прорывом. 

Свёрточные сети были придуманы при анализе части мозга, отвечающей за зрение. Оказывается, что соответствующие нейроны отвечают за абстрактные фичи в сигнале.

Крижевски не сделал ничего принципиально нового. Он взял эту архитектуру, которая была придумана ещё в девяностых, и нашел способ обучить её быстро — то есть на GPU.

![CNN](https://habrastorage.org/files/c36/bc5/b99/c36bc5b99dc14342b156fa742b285418.png)

# Представление изображений

Изображения будем представлять.

Хоть в случае черно-белых картинок (MNIST) первое измерение всегда равно единице, все равно будем.

Когда говорят «изображение», представляйте не прямоугольник, а параллелепипед, высотой которого будет размер каналов.

# Свёртка

Суть у них такая: давайте введем такую функцию, как **ядро** (англ. **kernel**). Он

1. Разобьем искомый паралеллелепипед на сколько-то одинаковых параллелепипедов. Они могут пересекаться.
2. Каждый из них развернем в вектор.
3. Каждый вектор и каждый 
4. Положим то, что получилось, в новый параллелепипед.
5. Посчитаем для кажой ячейки какую-нибудь нелинейность.

![conv1](images/conv1.png)

Можно добавить ещё такую вариацию:

**Паддинг**. Padding — это когда мы добавляем фиктивные граничные ячейки, которые заполняем нулями

**Страйды**. Stride — это насколько нужно сдвигать квадратики.

![conv1](images/conv2.png)

# Пулинг

Суть такая же, только в качестве ядра берется просто максимум, а между фильтрами (каналами) взаимодействия нет. Какая у этого интерпретация? Каждая ячейка хранит в себе некоторую оценку уверенности, что объект тут находится.

Увернность на следующем слое можно представить как 

MaxPooling используется для downsampling-а.

Пулинг это агрессивный даунсемплинг — он теряет информацию. Его критикуют, но он работает.

Вообще, свёртки есть не только для изображений. Иногда их применяют для анализа текстов или всяких геномов (одномерная свёртка) или 3д-изображений (всякие КТ, МРТ и прочие рентгены). Работают они примерно так же.

Тут можно много о чем подумать: сколько параметров будет в ядре, какая будет форма выходных данных, сколько оно будет считаться. Фреймворки классные, фреймворки делают это за нас.

# Почему это работает

Если определенным образом визуализировать те пиксели, которые больше всего триггерят конкретные нейроны, то можно понять, что они учат какие-то все более и более абстратные фичи.

Весов получается **мало**, так как они будут пошарены. Параметров меньше, каждый отвечает не только за конкретный пиксель — сети сложнее переобучиться.

![cat pic](images/cat.png)

# Свёртки

Фундаментальное отличие свёрточного слоя от полносвязного в том, что свёртки учат локальные паттерны в данных.

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

Здесь нам уже не надо их разворачивать в один вектор, так как нам важна пространственная информация.

In [0]:
from keras.datasets import mnist

(X, y), _ = mnist.load_data()

y = keras.utils.to_categorical(y, 10)
X = X.astype('float32')
X /= 255

# Сейчас X.shape это (60000, 28, 28)
# Формат Keras требует для сверток четырехмерный тензор, последним измерением которого идет канал
X = X.reshape(-1, 28, 28, 1)

In [0]:
model = Sequential([
    # channels  kernel_size
    Conv2D(8, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    Conv2D(16, (3, 3), activation='relu'), 
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.2),
    Flatten(), # разглаживает в один большой вектор
    Dense(128, activation='relu'), # Обычно такой добавляют в самый конец
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 26, 26, 8)         80        
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 24, 24, 16)        1168      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 12, 12, 16)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 12, 12, 16)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               295040    
_________________________________________________________________
dropout_13 (Dropout)         (None, 128)               0         
__________

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, batch_size=16, epochs=1, validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 64s - loss: 0.1032 - acc: 0.9694 - val_loss: 0.0461 - val_acc: 0.9867


Вот тут есть инструкция, как получить 99.5%, если прикрутить и зафайнтюнить все свистоперделки: https://www.kaggle.com/adityaecdrid/mnist-with-keras-for-beginners-99457

# Синтезируем данные

**Аугментацией** называется процесс получения новых данных из имеющихся. Это часто (особенно в компьютерном зрении) позволяет улучшить качество модели, не используя дополнительных данных.

Можно попробовать добавить следующие преобразования, которые с какой-то вероятностью будут использоваться во время обучения:

* Поворот на малый угол.
* Добавление шума.
* Обрезание границ и последующее растяжение до исходного размера.
* Горизонтальное отражение (но в нашем случае оно вредно).
* Смещение на небольшое расстояние.

Понятно, что лейбл эти преобразования изменить не должны.

Почти во всех фреймворках есть какой-то декларативный интерфейс для аугментации.

![dog](images/dog.jpeg)

In [0]:
from keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest')

gen.fit(X)

/home/sergey/.local/lib/python3.5/site-packages/keras/preprocessing/image.py:648: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (60000, 28, 28, 1) (1 channels).
  ' (' + str(x.shape[self.channel_axis]) + ' channels).')


In [0]:
model.fit_generator(gen.flow(X, y, batch_size=32),
                    epochs = epochs, validation_split=0.1)

# Как работать с внешними картинками

In [0]:
from keras.preprocessing.image import load_img, img_to_array
from os import ???

# Transfer Learning

Суть в том, чтобы использовать претрейненые слои с других нейросетей, обучавшихся очень долго на очень дорогом оборудовании.

Соревнования по CV выигрываются именно так.

![transfer](images/transfer.png)